In [11]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

start_time = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)
    
DCBS_DF = DCBS_DataFrame
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

B_DF = DTBS_DataFrame['B']

# 运行这个文件之前必须先去运行一下DCBS_toDataFrame文件，这样跑出来的结果才是最新的
# 计算所有转债每天的涨跌幅，并找出涨幅大于前N个交易日的q分位数涨跌幅的均值的转债，用于每天2点选出符合每日热门+双低策略的转债

In [12]:
# DataFrame新增列用于存放前一天收盘价、涨跌幅
# print(DCBS_DF.info())
DCBS_DF.sort_values(by = ['bond_code','date'], inplace=True, ascending=[True,True])
DCBS_DF['lastday_dp'] = DCBS_DF.groupby('bond_code')['dp'].shift(1)
DCBS_DF['change'] = DCBS_DF['dp'] / DCBS_DF['lastday_dp'] - 1
print(DCBS_DF)

# 设定选择区间为20天，得到低价中点与低价程度两列，利用滑动平均实现
DCBS_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = DCBS_DF.groupby('bond_code')
n = 20

# 两种不同的计算方式，一种是计算前20天收盘价的最高价、最低价（不含当天），一种是计算前20天收盘价的最高价、最低价（含当天）
# A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].max(), raw=True).reset_index(level=0, drop=True)
# A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].min(), raw=True).reset_index(level=0, drop=True)

DCBS_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).max().reset_index(level=0, drop=True)
DCBS_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).min().reset_index(level=0, drop=True)


DCBS_DF['low_price_level'] = DCBS_DF['dp'] / DCBS_DF['lowest_price_n_days'] - 1 
DCBS_DF['low_price_midpoint'] = (DCBS_DF['highest_price_n_days'] + DCBS_DF['lowest_price_n_days']) / 2

DCBS_DF['dp_median'] = DCBS_DF.groupby('date')['dp'].transform('median')
print(DCBS_DF)

                      bond_code        date    cpr       dp       dl    ytm  \
id                                                                            
110043.SH,2023-04-27  110043.SH  2023-04-27  11.78  112.976  124.756  -8.10   
110043.SH,2023-04-28  110043.SH  2023-04-28  12.25  113.247  125.497  -8.43   
110043.SH,2023-05-04  110043.SH  2023-05-04  11.49  114.459  125.949  -9.96   
110043.SH,2023-05-05  110043.SH  2023-05-05  11.91  114.298  126.208  -9.81   
110043.SH,2023-05-08  110043.SH  2023-05-08  11.45  116.600  128.050 -12.42   
...                         ...         ...    ...      ...      ...    ...   
128144.SZ,2023-08-30  128144.SZ  2023-08-30  51.26  116.030  167.290  -0.69   
128144.SZ,2023-08-31  128144.SZ  2023-08-31  50.61  115.400  166.010  -0.53   
128144.SZ,2023-09-01  128144.SZ  2023-09-01  50.25  115.812  166.062  -0.63   
128144.SZ,2023-09-04  128144.SZ  2023-09-04  48.63  117.000  165.630  -0.93   
128144.SZ,2023-09-05  128144.SZ  2023-09-05  46.19  

In [13]:
from datetime import date

day_list  = sorted(list(set(DCBS_DF['date'])))
print(day_list)

"""
设置分位点，时间参数（这些参数可以后期调整）；
q_change（判定债券是否热门的分位点），q_dl（判定债券双低是否较低的分位点）
n(用前n天作为一个周期看债券之前是否热门),m（一个周期内需要几天涨幅较大超过分位点才能认为这只债券之前是热门债券）
"""

q_change = 0.95
n = 10
q_dl = 0.3
m = 1 

# 默认结束日期是day_list列表中的最后一天，可以手动修改没必要
# 使用数据从'2023-04-27'-'2023-07-13',实际测试时间从'2023-5-18'-'2023-07-13',因为需要测试日的前10个交易日的数据去观察这个债券之前是否热门
start_index = day_list.index('2023-04-27')
end_index = day_list.index(day_list[-1])

day_list = day_list[start_index:end_index+1]
test_day_list = day_list[n+1:]

# 最后需要展示的列
need_columns = ['date' ,'bond_code' ,'cn' ,'sc' ,'sn','dp','change', 'dl','cpr','bl','yl','过去一段时间是热门转债的天数', '前n天内所有热门的日期']
empty_df_select = pd.DataFrame(columns = need_columns)


# 挑选出符合热门+双低的转债
DCBS_DF['quantile_change'] = DCBS_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))
DCBS_DF['quantile_dl'] = DCBS_DF.groupby('date')['dl'].transform(lambda x: x.dropna().quantile(q_dl))
DCBS_DF['当日是否属于热门转债'] = (DCBS_DF['change'] >= DCBS_DF['quantile_change']).astype(int)
DCBS_DF['当日是否双低较小'] = (DCBS_DF['dl'] <= DCBS_DF['quantile_dl']).astype(int)


DCBS_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = DCBS_DF.groupby('bond_code')
DCBS_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
DCBS_DF['是否属于过去一段时间内的热门转债'] = (DCBS_DF['过去一段时间是热门转债的天数'] >= m).astype(int)


# 这里的功能是找出前n天所有热门的日期，这个步骤会耗费大量运算资源，非必要不输出这一条信息
DCBS_DF['last_n_date'] = DCBS_DF.groupby('bond_code')['date'].shift(n)

def filter_dates(group):
    group['前n天内所有热门的日期'] = group.apply(
        lambda row: '、'.join(group[
            (group['当日是否属于热门转债'] == 1)&
            (group['date'] >= row['last_n_date'])& 
            (group['date'] < row['date'])              
        ]['date'].tolist()),
        axis=1
    )
    return group

DCBS_DF['前n天内所有热门的日期'] = DCBS_DF.groupby('bond_code', group_keys=True).apply(filter_dates).reset_index(level=0, drop=True)['前n天内所有热门的日期']
# 这里的功能是找出前n天所有热门的日期，这个步骤会耗费大量运算资源，非必要不输出这一条信息

empty_df_select = DCBS_DF[(DCBS_DF['是否属于过去一段时间内的热门转债']==1) & 
                          (DCBS_DF['当日是否双低较小']==1) & 
                          (DCBS_DF['date'] >= test_day_list[0]) &
                          (DCBS_DF['date'] <= test_day_list[-1])][need_columns].copy()
empty_df_select.sort_values(by = ['date','change','dl'], inplace=True, ascending=[True,False,True])
# print(empty_df_select)


# 赎回和下修的指标只有5点更新了数据库之后才能导出来，否则只能使用昨天的数据 ，获取qs,xx等指标
both_exist_date_list = list(set(empty_df_select['date']) & set(A_DF['date']))
different_date_list = list(set(empty_df_select['date']) - set(A_DF['date']))

if len(different_date_list) > 0:
    temp1_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')
    
    last_day_A_DF = A_DF[A_DF['date']==sorted(list(set(A_DF['date'])))[-1]].copy()
    temp2_df = pd.merge(empty_df_select[empty_df_select['date'].isin(different_date_list)], last_day_A_DF[['bond_code', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code'], how='left')
    
    temp_df = pd.concat([temp1_df, temp2_df])    
else:
    temp_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')

temp_df.reset_index(drop=True)
temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']] = temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']].astype('Int64')

temp_df = pd.merge(temp_df,B_DF[['bond_code','cat1','cat2','risk_warning','hot_concept']],on='bond_code',how='left')

empty_df_select = temp_df.copy()


"""保存到EXCEL的时候重命名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'cn' : '可转债名称',
                                'sc' : '正股代码',
                                'sn' : '正股名称',
                                'dp' : '价格（2点数据计算）',
                                'cpr' : '转股溢价率（两点数据计算）',
                                'dl' : '双低（两点数据计算）',
                                'change':'涨跌幅（两点数据计算）',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'risk_warning' : '风险提示',
                                'hot_concept' : '热门概念',
                                'cat1':'申万一级',
                                'cat2':'申万二级'},inplace=True)

empty_df_select.to_excel(day_list[end_index]+' 热门可转债+双低.xlsx',index=False)
empty_df_select.loc[empty_df_select['日期']==day_list[end_index]].to_excel(day_list[end_index]+' 热门可转债+双低(只有今天的数据).xlsx',index=False)

print(day_list[end_index], '（双低的前30%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dl']),0.3),2))
print(day_list[end_index], '（双低的前50%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dl']),0.5),2))
print(day_list[end_index], '（价格的前30%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dp']),0.3),2))
print(day_list[end_index], '（价格的前50%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dp']),0.5),2))

# 避免名字发生变化，使用bond_code去判断是否新增更准确
new_code = list(set(empty_df_select.loc[empty_df_select['日期']==day_list[end_index],'可转债代码']) - set(empty_df_select.loc[empty_df_select['日期']==day_list[end_index-1],'可转债代码']))
new_cn = list(empty_df_select.loc[(empty_df_select['日期']==day_list[end_index]) & (empty_df_select['可转债代码'].isin(new_code)),'可转债名称'])
print('今日新增的符合热点-双低策略的转债：', '、'.join(new_cn))

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25', '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10', '2023

In [4]:
# ----------------------------------------------------------------------------------------------------------------------
# --------------------------------------------------- 分页，便于阅读 ---------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------------
# --------------------------------------------------- 分页，便于阅读 ---------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------------

In [5]:
from datetime import date

day_list  = sorted(list(set(DCBS_DF['date'])))
print(day_list)

"""
设置分位点，时间参数（这些参数可以后期调整）；
q_change（判定债券是否热门的分位点），q_dl（判定债券双低是否较低的分位点）
n(用前n天作为一个周期看债券之前是否热门),m（一个周期内需要几天涨幅较大超过分位点才能认为这只债券之前是热门债券）
"""

q_change = 0.95
n = 10
q_dl = 0.3
m = 1
q_low_price_level = 0.3

# 默认结束日期是day_list列表中的最后一天，可以手动修改没必要
# 使用数据从'2023-04-27'-'2023-07-13',实际测试时间从'2023-5-18'-'2023-07-13',因为需要测试日的前10个交易日的数据去观察这个债券之前是否热门
start_index = day_list.index('2023-04-27')
end_index = day_list.index(day_list[-1])

day_list = day_list[start_index:end_index+1]
test_day_list = day_list[n+1:]

# 最后需要展示的列
need_columns = ['date' ,'bond_code' ,'cn' ,'sc' ,'sn','dp','change', 'dl','cpr','bl','yl','low_price_level']
empty_df_select = pd.DataFrame(columns = need_columns)


# 挑选出符合热门+双低的转债
DCBS_DF['quantile_change'] = DCBS_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))
DCBS_DF['quantile_dl'] = DCBS_DF.groupby('date')['dl'].transform(lambda x: x.dropna().quantile(q_dl))
DCBS_DF['当日是否属于热门转债'] = (DCBS_DF['change'] >= DCBS_DF['quantile_change']).astype(int)
DCBS_DF['当日是否双低较小'] = (DCBS_DF['dl'] <= DCBS_DF['quantile_dl']).astype(int)
DCBS_DF['quantile_low_price_level'] = DCBS_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(q_low_price_level))

DCBS_DF['当日是否低价程度较小'] = (DCBS_DF['low_price_level'] <= DCBS_DF['quantile_low_price_level']).astype(int)

DCBS_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = DCBS_DF.groupby('bond_code')
DCBS_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
DCBS_DF['是否属于过去一段时间内的热门转债'] = (DCBS_DF['过去一段时间是热门转债的天数'] >= m).astype(int)

# print(DCBS_DF)

empty_df_select = DCBS_DF[(DCBS_DF['是否属于过去一段时间内的热门转债']==1) & 
                          # (DCBS_DF['当日是否双低较小']==1) & 
                          # (DCBS_DF['当日是否低价程度较小']==1) &
                          (DCBS_DF['dp'] >= 110) &
                          (DCBS_DF['dp'] <= 130) &
                          (DCBS_DF['date'] >= test_day_list[0]) &
                          (DCBS_DF['date'] <= test_day_list[-1])][need_columns].copy()
empty_df_select.sort_values(by = ['date','change','dl'], inplace=True, ascending=[True,False,True])
# print(empty_df_select)


# 赎回和下修的指标只有5点更新了数据库之后才能导出来，否则只能使用昨天的数据 ，获取qs,xx等指标
both_exist_date_list = list(set(empty_df_select['date']) & set(A_DF['date']))
different_date_list = list(set(empty_df_select['date']) - set(A_DF['date']))

if len(different_date_list) > 0:
    temp1_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')
    
    last_day_A_DF = A_DF[A_DF['date']==sorted(list(set(A_DF['date'])))[-1]].copy()
    temp2_df = pd.merge(empty_df_select[empty_df_select['date'].isin(different_date_list)], last_day_A_DF[['bond_code', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code'], how='left')
    
    temp_df = pd.concat([temp1_df, temp2_df])    
else:
    temp_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')

temp_df.reset_index(drop=True)
temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']] = temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']].astype('Int64')

empty_df_select = temp_df.copy()


"""保存到EXCEL的时候重命名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'cn' : '可转债名称',
                                'sc' : '正股代码',
                                'sn' : '正股名称',
                                'dp' : '价格（2点数据计算）',
                                'cpr' : '转股溢价率（两点数据计算）',
                                'dl' : '双低（两点数据计算）',
                                'change':'涨跌幅（两点数据计算）',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）'},inplace=True)

empty_df_select.to_excel(day_list[end_index]+' 热门可转债(测试).xlsx',index=False)
empty_df_select.loc[empty_df_select['日期']==day_list[end_index]].to_excel(day_list[end_index]+' 热门可转债(测试，只有今天的数据).xlsx',index=False)

print(day_list[end_index], '（双低的前30%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dl']),0.3),2))
print(day_list[end_index], '（双低的前50%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dl']),0.5),2))
print(day_list[end_index], '（价格的前30%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dp']),0.3),2))
print(day_list[end_index], '（价格的前50%分位数）：', round(np.quantile(list(DCBS_DF[DCBS_DF['date'] == day_list[end_index]]['dp']),0.5),2))

# 避免名字发生变化，使用bond_code去判断是否新增更准确
new_code = list(set(empty_df_select.loc[empty_df_select['日期']==day_list[end_index],'可转债代码']) - set(empty_df_select.loc[empty_df_select['日期']==day_list[end_index-1],'可转债代码']))
new_cn = list(empty_df_select.loc[(empty_df_select['日期']==day_list[end_index]) & (empty_df_select['可转债代码'].isin(new_code)),'可转债名称'])
# print('今日新增的符合热点-低价策略的转债：', new_cn)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")


['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25', '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10', '2023

In [6]:
# 某一天根据当天需求弄出来的
DCBS_DF['quantile_low_price_level_50'] = DCBS_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(0.5))
DCBS_DF['quantile_low_price_level_30'] = DCBS_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(0.3))
DCBS_DF['quantile_low_price_level_25'] = DCBS_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(0.25))
DCBS_DF['quantile_low_price_level_20'] = DCBS_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(0.2))

last_20_days = sorted(list(set(DCBS_DF['date'])))[-20:]
print(last_20_days)


avg_50 = list(DCBS_DF[(DCBS_DF['date'].isin(last_20_days)) & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_50'])
print(avg_50)
avg_50 = sum(avg_50)/ len(avg_50)

avg_30 = list(DCBS_DF[(DCBS_DF['date'].isin(last_20_days)) & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_30'])
print(avg_30)
avg_30 = sum(avg_30)/ len(avg_30)

avg_25 = list(DCBS_DF[(DCBS_DF['date'].isin(last_20_days)) & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_25'])
print(avg_25)
avg_25 = sum(avg_25)/ len(avg_25)

avg_20 = list(DCBS_DF[(DCBS_DF['date'].isin(last_20_days)) & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_20'])
print(avg_20)
avg_20 = sum(avg_20)/ len(avg_20)

print('50%   ',avg_50)
print('30%   ',avg_30)
print('25%   ',avg_25)
print('20%   ',avg_20,'\n')

print('2023-08-10  50% ', float(DCBS_DF[(DCBS_DF['date']=='2023-08-10') & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_50']))
print('2023-08-10  30% ', float(DCBS_DF[(DCBS_DF['date']=='2023-08-10') & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_30']))
print('2023-08-10  25% ', float(DCBS_DF[(DCBS_DF['date']=='2023-08-10') & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_25']))
print('2023-08-10  20% ', float(DCBS_DF[(DCBS_DF['date']=='2023-08-10') & (DCBS_DF['bond_code']=='110043.SH')]['quantile_low_price_level_20']))


['2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11', '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17', '2023-08-18', '2023-08-21', '2023-08-22', '2023-08-23', '2023-08-24', '2023-08-25', '2023-08-28', '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-01', '2023-09-04']
[0.014697932152993642, 0.012554724913840909, 0.012879510033096842, 0.010746371933459398, 0.005335877862595595, 0.003367636653500705, 0.004422512148727575, 0.006875923190546418, 0.009478549005929482, 0.005029142381348883, 0.0, 0.00455038202918967, 0.004588566402012106, 0.0013170040652954462, 0.006171953495048177, 0.016494030898876355, 0.018476851366317604, 0.01716660791434732, 0.019409528313899527, 0.02449557522123902]
[0.00793581809331756, 0.0057676808269135444, 0.005547235860572774, 0.0040549367215704235, 0.0, 0.0, 0.0008274199652825497, 0.003168472630031438, 0.00497397653555862, 0.001237926670668975, 0.0, 0.00156810278495465, 0.0017028393294979294, 0.0, 0.00196498599811723, 0.010775305032932003, 0.0113016895

In [7]:
# 临时需求，2023-08-21

from datetime import date

day_list  = sorted(list(set(DCBS_DF['date'])))
print(day_list)

"""
设置分位点，时间参数（这些参数可以后期调整）；
q_change（判定债券是否热门的分位点），q_dl（判定债券双低是否较低的分位点）
n(用前n天作为一个周期看债券之前是否热门),m（一个周期内需要几天涨幅较大超过分位点才能认为这只债券之前是热门债券）
"""

q_change = 0.95
n = 10
q_dl = 0.3
m = 1
q_low_price_level = 0.3

# 默认结束日期是day_list列表中的最后一天，可以手动修改没必要
# 使用数据从'2023-04-27'-'2023-07-13',实际测试时间从'2023-5-18'-'2023-07-13',因为需要测试日的前10个交易日的数据去观察这个债券之前是否热门
start_index = day_list.index('2023-04-27')
end_index = day_list.index(day_list[-1])

day_list = day_list[start_index:end_index+1]
test_day_list = day_list[n+1:]

# 最后需要展示的列
need_columns = ['date' ,'bond_code' ,'cn' ,'sc' ,'sn','dp','change','ytm', 'dl','cpr','bl','yl','low_price_level']
empty_df_select = pd.DataFrame(columns = need_columns)


# 挑选出符合热门+双低的转债
DCBS_DF['quantile_change'] = DCBS_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))
DCBS_DF['quantile_dl'] = DCBS_DF.groupby('date')['dl'].transform(lambda x: x.dropna().quantile(q_dl))
DCBS_DF['当日是否属于热门转债'] = (DCBS_DF['change'] >= DCBS_DF['quantile_change']).astype(int)
DCBS_DF['当日是否双低较小'] = (DCBS_DF['dl'] <= DCBS_DF['quantile_dl']).astype(int)
DCBS_DF['quantile_low_price_level'] = DCBS_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(q_low_price_level))

DCBS_DF['当日是否低价程度较小'] = (DCBS_DF['low_price_level'] <= DCBS_DF['quantile_low_price_level']).astype(int)

DCBS_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = DCBS_DF.groupby('bond_code')
DCBS_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
DCBS_DF['是否属于过去一段时间内的热门转债'] = (DCBS_DF['过去一段时间是热门转债的天数'] >= m).astype(int)
# print(DCBS_DF)

empty_df_select = DCBS_DF[(DCBS_DF['是否属于过去一段时间内的热门转债']==1) &
                          (DCBS_DF['ytm'] >= 1) &
                          (DCBS_DF['date'] >= test_day_list[0]) &
                          (DCBS_DF['date'] <= test_day_list[-1])][need_columns].copy()
empty_df_select.sort_values(by = ['date','change','dl'], inplace=True, ascending=[True,False,True])
# print(empty_df_select)


# 赎回和下修的指标只有5点更新了数据库之后才能导出来，否则只能使用昨天的数据 ，获取qs,xx等指标
both_exist_date_list = list(set(empty_df_select['date']) & set(A_DF['date']))
different_date_list = list(set(empty_df_select['date']) - set(A_DF['date']))

if len(different_date_list) > 0:
    temp1_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')
    
    last_day_A_DF = A_DF[A_DF['date']==sorted(list(set(A_DF['date'])))[-1]].copy()
    temp2_df = pd.merge(empty_df_select[empty_df_select['date'].isin(different_date_list)], last_day_A_DF[['bond_code', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code'], how='left')
    
    temp_df = pd.concat([temp1_df, temp2_df])    
else:
    temp_df = pd.merge(empty_df_select[empty_df_select['date'].isin(both_exist_date_list)], A_DF[['bond_code', 'date', 'qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']],on=['bond_code','date'], how='left')

temp_df.reset_index(drop=True)
temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']] = temp_df[['qs', 'qs15', 'qs30', 'xx', 'xx15', 'xx30']].astype('Int64')

empty_df_select = temp_df.copy()


"""保存到EXCEL的时候重命名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'cn' : '可转债名称',
                                'sc' : '正股代码',
                                'sn' : '正股名称',
                                'dp' : '价格（2点数据计算）',
                                'cpr' : '转股溢价率（两点数据计算）',
                                'dl' : '双低（两点数据计算）',
                                'change':'涨跌幅（两点数据计算）',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）'},inplace=True)


print(empty_df_select.loc[empty_df_select['日期']==day_list[end_index]])

['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25', '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10', '2023